In [1]:
import csv
import os
import numpy as np

#### Convertir .txt à .csv
Ça sera plus facile de gerer la donnée 

In [2]:
def txt_to_csv(input_path, output_path):
    rows = []

    with open(input_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # Remove leading '#' if present
            if line.startswith('#'):
                line = line[1:]

            # Split on any whitespace
            parts = line.split()
            rows.append(parts)

    # Write CSV
    with open(output_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

In [3]:
txt_to_csv("../data/raw/raw.txt", "../data/raw/raw.csv")

#### Normaliser la donnée
Sinon c'est inutilisable

In [4]:
import csv
import os
import numpy as np

def normalize_csv(input_csv_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # --- Load raw rows ---
    with open(input_csv_path, "r") as f:
        reader = csv.reader(f)
        raw_rows = [row for row in reader]

    # Detect headers: non-numeric cells → header row
    def is_header(row):
        for cell in row:
            try:
                float(cell)
            except ValueError:
                return True
        return False

    header_rows = []
    data_rows = []

    for row in raw_rows:
        stripped = [c.strip() for c in row if c.strip() != ""]
        if len(stripped) == 0:
            continue

        if is_header(stripped):
            header_rows.append(stripped)
        else:
            data_rows.append(stripped)

    # Ensure consistent length
    lengths = {len(r) for r in data_rows}
    if len(lengths) != 1:
        raise ValueError(
            f"Inconsistent row lengths detected: {lengths}."
        )

    # Convert to float matrix
    data = np.array([[float(x) for x in row] for row in data_rows], dtype=float)

    # Compute stats
    means = data.mean(axis=0)
    stds = data.std(axis=0)
    stds = np.where(stds == 0, 1.0, stds)

    # Normalize
    normalized = (data - means) / stds

    # -------- SPLIT INTO TRAIN (95%) AND TEST (5%) --------
    n = len(normalized)
    split_idx = int(n * 0.95)

    train_data = normalized[:split_idx]
    test_data = normalized[split_idx:]

    # Save train.csv
    train_path = os.path.join(output_dir, "train.csv")
    with open(train_path, "w", newline="") as f:
        writer = csv.writer(f)
        for hr in header_rows:
            writer.writerow(hr)
        for row in train_data:
            writer.writerow(row.tolist())

    # Save test.csv
    test_path = os.path.join(output_dir, "test.csv")
    with open(test_path, "w", newline="") as f:
        writer = csv.writer(f)
        for hr in header_rows:
            writer.writerow(hr)
        for row in test_data:
            writer.writerow(row.tolist())

    # Save the full normalized file (optional)
    normalized_path = os.path.join(output_dir, "normalized.csv")
    with open(normalized_path, "w", newline="") as f:
        writer = csv.writer(f)
        for hr in header_rows:
            writer.writerow(hr)
        for row in normalized:
            writer.writerow(row.tolist())

    # Save stats
    stats_path = os.path.join(output_dir, "normalization_stats.csv")
    with open(stats_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["column_index", "mean", "std"])
        for i, (m, s) in enumerate(zip(means, stds)):
            writer.writerow([i, m, s])

    print("Done.")
    print("Saved train file to:", train_path)
    print("Saved test file to:", test_path)
    print("Saved full normalized file to:", normalized_path)
    print("Saved stats file to:", stats_path)


In [5]:
normalize_csv("../data/raw/raw.csv", "../data/raw_normalized")

Done.
Saved train file to: ../data/raw_normalized/train.csv
Saved test file to: ../data/raw_normalized/test.csv
Saved full normalized file to: ../data/raw_normalized/normalized.csv
Saved stats file to: ../data/raw_normalized/normalization_stats.csv
